In [14]:
import netket as nk
import netket.nn as nnx

import numpy as np
from PIL.GimpGradientFile import curved

from scipy.sparse.linalg import eigsh

from tqdm import tqdm
from functools import partial

import jax
import jax.numpy as jnp

from flax import linen as nn # flax is library that brings a more pytorch-like API to JAX
import optax # optax is a library that provides optimizers and learning rate schedules

import matplotlib.pyplot as plt

In [15]:
# defining the hamiltonian

chain_length = 20
coupling_strength = 1.0
magnetic_field = 1.0

g = nk.graph.Hypercube(length=chain_length, n_dim=1, pbc=True)
hi = nk.hilbert.Spin(s=1/2, N=g.n_nodes)

h = nk.operator.Ising(hi, g, h=magnetic_field, J=coupling_strength, dtype=float)

In [16]:
# defining the ansatz

class JasShort(nnx.Module):
    def __init__(self, *, rngs: nnx.Rngs):
        self.j1 = nnx.Param(0.01 * jax.random.normal(rngs.params(), (1,)), dtype=float)
        self.j2 = nnx.Param(0.01 * jax.random.normal(rngs.params(), (1,)), dtype=float)

    def __call__(self, x: jax.Array):
        corr1 = x * jnp.roll(x, -1, axis=-1)
        corr2 = x * jnp.roll(x, -2, axis=-1)
        return jnp.sum(self.j1 * corr1 + self.j2 * corr2, axis=-1)

/Users/Tonni/miniforge3/envs/dlnn2/lib/python3.9/site-packages/netket/nn/__init__.py:90: FutureWarning: 

          `nk.nn.Module` is deprecated. Use `flax.linen.Module` directly instead.

If you imported `flax.linen as nn`, as is customary to do, you can use `nn.Module` directly. There are no functionality changes.

  _warn_deprecation(


AttributeError: module netket.nn has no attribute Rngs